<a href="https://colab.research.google.com/github/kocurvik/edu/blob/master/PNNPPV/notebooky/cv02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2. cvičenie** - Linárna algebra s NumPy, Plne prepojená sieť

V tomto notebooku si prejdeme operácie lineárnej algebry v NumPy. Tie potom využijeme aby sme naprogramovali plne prepojenú sieť.

#Lineárna Algebra s NumPy

Na implementáciu základnej plne prepojenej siete budeme potrebovať maticové násobenie. V skutočnosti budeme násobiť tenzory, ale najprv si precvičíme násobenie matíc.

## Maticové násobenie
Najdôležitejšia operácia, ktorú budeme potrebovať je maticové násobenie. Najprv definícia:

Nech $\mathbf{A} \in \mathbb{R}^{m\times n}, \mathbf{B} \in \mathbb{R}^{n \times l}, \mathbf{C} \in \mathbb{R}^{m \times l} $ potom $\mathbf{A}\mathbf{B} = \mathbf{C} \iff (\forall i \in \hat{m})(\forall j \in \hat{l})(c_{i,j} = \sum_{k=1}^{n} a_{i,k} \cdot b_{k,j})$ 

K definícii je dobré si pripomenúť, že prvý index označuje riadok a druhý index označuje stĺpec.

### Úloha 1 (Na tabuľu)

Spočítajte súčiny $\mathbf{A}\mathbf{B}$, $\mathbf{B}\mathbf{C}$, $\mathbf{C}\mathbf{B}$

$\mathbf{A} = \begin{bmatrix} 
3 & 5 & -1 \\
2 & -4 & 2
\end{bmatrix}$

$\mathbf{B} = \begin{bmatrix} 
5 & 2 & 1 \\
-6 & 5 & 2 \\
3 & 4 & -1
\end{bmatrix}$

$\mathbf{C} = \begin{bmatrix} 
4 & -4 & 3 \\
-6 & -3 & 4 \\
-1 & 1 & 0
\end{bmatrix}$

V numpy násobíme matice pomocou príkazu np.matmul(a,b), alebo pomocou operátora @. Dá sa použiť aj np.dot. Rozdiel medzi multiply a dot je v broadcastingu. Otestujte aj súčin $\mathbf{BC}$ a $\mathbf{CB}$

*Pozn:* Vektory môžeme považovať za matice s jednou dimenziou jedna. Podľa toho či je stĺpcový, alebo riadkový. Formálne to mení možnosti násobenia. Toto ale v NumPy nieje tak vždy. Pri násobení sa tak arrays ktoré majú len jeden rád, tj. len(np.shape) == 1 upravia podľa toho čo sa viac hodí, či riadkový, alebo stĺpcový vektor.

In [1]:
import numpy as np
a = np.array([[3,5,-1],[2,-4,2]])
b = np.array([[5,2,1],[-6,5,2],[3,4,-1]])
print(np.matmul(a,b))
d = a @ b
print(d)
v = np.array([10,20,30])
print(v.shape)
u = np.array([5,25])
print(u.shape)
print(a @ v)
print(u @ a)


[[-18  27  14]
 [ 40  -8  -8]]
[[-18  27  14]
 [ 40  -8  -8]]
(3,)
(2,)
[100   0]
[ 65 -75  45]


V prípade, že máme jeden tenzor vyššieho rádu ako 2, tak matmul ich bude brať ako zoznam matíc. Ak potrebujeme robiť exotickejšie operácie, tak môžeme použiť príkaz np.einsum.

## Skalárne násobenie

Skalárom môžeme jednoducho násobiť matícu. To sa robí pomocou np.multiply, alebo operátorom *. Ak nenásobíme skalárom ale tenzorom, tak dôjde k násobeniu po elementoch s príšlušným broadcastingom. 

In [2]:
print(a * d)
print(5 * a)

[[-54 135 -14]
 [ 80  32 -16]]
[[ 15  25  -5]
 [ 10 -20  10]]


## Transponované matice

Definícia: Nech $\mathbf{A} \in \mathbb{R}^{m,n}$ potom $\mathbf{A}^T \in \mathbb{R}^{n,m}$ je jej transponovaná matica $\iff (\forall i \in \hat{m})(\forall j \in \hat{n})(a_{i,j} = a^T_{j,i})$ 

V numpy jednoducho voláme metódu np.array .T, alternatívne možeme použiť funkciu np.transpose. Opäť platí, že rovnako môžeme postupovať aj v prípade vektorov zapísaných ako matice.


In [6]:
print(a.T)
print(np.transpose(a))

r = np.array([[100,10,1]])
print(r.shape)

c = np.array([[3],[7],[8]])
print(c.shape)

print(a @ c)
print(a @ r.T)



[[ 3  2]
 [ 5 -4]
 [-1  2]]
[[ 3  2]
 [ 5 -4]
 [-1  2]]
(1, 3)
(3, 1)
[[36]
 [-6]]
[[349]
 [162]]


# Plne prepojená neurónová sieť
Neurónová sieť je biologicky inšpirovaný model pre realizáciu výpočtov rôznych funkcií. Dnes ju budeme využívať na ako klasifikátor. 

Sieť obecne modelujeme ako orientovaný graf s ohodnotenými hranami ktorého vrcholy sú tzv. neuróny. Každý neurón má svoju aktiváciu, ktorá sa počíta na základe aktivácii neurónov s ktorými je prepojený. Táto aktivácia zas ovplivňuje ďalšie neuróny. Najobecnejšie tak môžeme popísať sieť ako:
$$a_p = f \left( \sum_{q \in p_{in}} w_{p,q} a_q + b_p \right) = f\left(z_p\right),$$

kde $a_p$ je aktivácia daného vrcholu, $w_{p,q}$ je váha vrcholu $q$ pre vrchol $p$, $b_p$ je prah vrcholu $p$, $f$ je aktivačná funckia, $z_p$ je zjednodušený zápis vstupu aktivačnej funkcie. V takomto zápise je však možné aby boli neuróny prepojené cyklicky, čo nechceme. Takisto nechceme, aby bola štruktúra siete komplikovaná. Preto zavedieme tzv. plne prepojenú neurónovú sieť, tiež označovaný ako Multi-Layer-Perceptron. Táto organizácia spočíva v tom, že každý neurón je v nejakej vrstve a každý neurón v jednej vrstve je prepojený s každým z predchádzajúcej.

![Plne prepojená sieť](https://raw.githubusercontent.com/kocurvik/edu/master/PNNPPV/supplementary/ntb_images/NN1.jpg)

Vyhodnotenie siete potom môžeme zapísať ako:

$$a_j^l = f \left( \sum_{k} a_k^{l-1} w_{k,j}^l + b_j^l \right) = f \left(z_j^l \right),$$

resp. vektorovo:

$$a^l = f \left( a^{l-1}w^l + b^l \right) =  f \left(z^l \right),$$

kde $a^l$ je **riadkový** vektor aktivácií, $w^l$ je matica váh tvaru $size(l-1) \times size(l)$, $a_l$ je **riadkový** vektor prahov, $f$ je skalárna aktivačná funkcia, ktorá je vo vektorovom prípade aplikovaná po elementoch, $z_l$ je taktiež riadkový vektor, ktorý použijeme na zjednodušenie výrazov. Horný index pri každom výraze značí, ku ktorej vrstve daný objekt partrí.

Pre lepšiu predstavu o indexácií viď obrázky:

![Indexy aktivácie a prahu](https://raw.githubusercontent.com/kocurvik/edu/master/PNNPPV/supplementary/ntb_images/activation_bias.jpg)
![Indexy váh](https://raw.githubusercontent.com/kocurvik/edu/master/PNNPPV/supplementary/ntb_images/weight.jpg)

*Pozn.:* Vektory by mohli byť aj stĺpcové (a matica tak transponovaná a násobenie v opačnom poradí), ale to by nám v NumPy skomplikovalo prácu kvôli broadcastingu.







### Aktivačné funkcie

Môžeme použiť rôzne aktivačné funkcie. Dnes ostaneme pri sigmoide, ale nabudúce si implementujeme aj ďalšie.

1. Sigmoid: $f(z) = \frac{1}{1 + e^{-z}}$
2. Tanh: $f(z) = tanh(z)$
3. ReLU: $f(z) = max(x,0)$
4. SoftPlus: $f(z) = ln(1 + e^z)$
5. LeakyReLU: $f(z) = max(x,ax), a \le 1$


### Interpretácia poslednej vrstvy

Zameriame sa na úlohu klasifikácie. Preto budeme interpretovať poslednú vrstvu v tomto zmysle. Ak chcem klasifikovať len do dvoch tried, tak nám stačí jeden neurón so sigmoidom. Ak je jeho hodnota menšia ako 0.5 klasifikujeme objekt ako prvú triedu a inak ako druhú.

Pri viacerých $n \gt 2$ triedach potrebujeme viacero neurónov môžeme použiť pravidlo: $k = {argmax}_{i \in \hat{n}}(a_i^L(x))$, kde $L$ je počet vrstiev, $x$ je vstupný vektor $n$ je počet tried a $k$ je trieda ktorú klasifikátor určil.

Trocha sofistikvanejší postup je použitie. Tzv. softmax vrstvy na konci namiesto bežnej aktivácie. $P(k|x) = \frac{e^{z^L_k(x)}}{\sum_{i \in \hat n} e^{z^L_i(x)}}$, kde $P(k|x)$ je pravdepodobnosť, že pre vstup $x$ je správna trieda $k$.


## Trénovanie

Model je síce pekný, ale je nutné nájsť také aktivácie aby robil to, čo chceme. V našom prípade je to klasifikácia do $n$ tried. Na trénovanie budeme potrebovať trénovacie dáta. Teda páry $\left(x, y\right)$, kde $x$ je vstupný vektor a $y$ je výstupný vektor označujúci správnu triedu. Aby sa nám to hodilo do interpretácie poslednej vrstvy, tak $y$ bude vektor, ktorý bude mať na $k$-tom mieste jednotku a všade inde nuly. Toto je tzv. **one-hot** kódovanie.

### Cenová funkcia

Trénovanie je podobné optimalizácii. Chceme nájsť také parametre siete (v našom prípade matice $w$ a vektory $b$), ktoré budú fungovať čo najlepšie pre náš problém. To sa budeme snažiť docieliť tým že budeme optimalizovať hodnotu tzv. ceny (loss function, cost function). Ideálne platí, že cím vyššia cena na nejakej množine, tým horšie na nej naša sieť funguje. Zároveň chceme aby bola cena pekne diferencovateľná. Dnes použijeme cenové funkcie pre $N$ párov $\left(x^i, y^i\right)$:

1. MSE: $C = \frac{1}{N} \sum_i ||a^L(x^i) - y^i||_2^2$, kde $||v||_2$ je L-2 norma vektoru $v$.
2. Cross-Entropy:  $C = -\frac{1}{N} \sum_{i,j} y_j^i ln\left(a_j^L(x)\right) + \left(1-y_j^i\right) ln\left(1 - a_i^L(x) \right)$
3. CE + Softmax: $C = \frac{1}{N} ln(z_y^L(x))$, kde $z_y^L(x)$ je ten element vektoru $z^L(x)$, pre ktorý je trieda správne. 


### Gradientný zostup

Na optimalizáciu použijeme jednoduché pravidlo gradientného zostupu s krokom $\eta$ pre parameter $p$ a cenovú funkciu $C$.

$$p := p - \eta \frac{\partial C}{\partial p}$$



### SGD

Na každý krok pri trénovaní vždy použijeme náhodnú podmnožinu z trénovacej množiny. Na ďalší krok ďalšiu atď. Toto má výhody v tom, že vnášame do procesu šum, ktorý nás vie dostať z lokálnych miním. Takisto miesta v parametrickom priestore kam sa algoritmus dostane majú často lepšie vlastnosti, čo sa týka generalizácie ako čistá optimalizácia. Navyše tým šetríme miesto v pamäti, čo je jedna zo základných.

Tento prístup sa volá **stochastic gradient descent (SGD)**.

*Pozn.:* Výberom minibatchu vlastne approximujeme priestor funkciu $C$ pre celú trénovaciu množinu. "Chyba" tejto aproximácie klesá približne $\sim \frac{1}{\sqrt{M}}$, kde $M$ je veľkosť minibatchu. Preto nieje úplne vhodné používať veľký minibatch, aj keď to pamäť dovoluje.

### Výpočet parciálnych derivácii
Teraz si odvodíme postup ako vypočítať analyticky parciálne derivácie pre našu plne prepojenú sieť.

Najprv si zavedieme pomocnú deriváciu.

$ d_i^l = \frac{\partial C}{\partial z_i^l}$

V závislosti na výbere $C$ platí, značka $\odot$ značí násobenie po elementoch (Hadamardov súčin)

pre MSE: $d_i^L = (a_i^L - y_i) \cdot f^{'} (z_i^L)$, 

pre CE: $d_i^L = (a_i^L - y_i)$

Vektorovo MSE: $d_i^L = (a^L - y) \odot f^{'} (z^L)$

Vektorovo CE: $d^L = a^L - y$

Na základe $d^{l+1}$ môžeme určiť $d^l$. Tento postup je tzv. **backpropagation**, keďže deriváciu (gradient) propagujeme v sieti opačným smerom ako pri doprednom výpočte.

$ d^l = \left(d^{l+1} \left( w^{l+1} \right)^T\right) \odot f^{'}(z^l)$

Dôkaz:

$z_j^{l+1} = \sum_k \left( f \left( z^l_k\right) w_{kj}^{l+1} \right) + b_j^{l+1}$

$\frac{\partial z_j^{l+1}}{\partial z_k^l} = f^{'} \left( z^l_k\right) w_{kj}^{l+1}$

$d_k^l = \frac{\partial C}{\partial z_k^l} = \sum_j \frac{\partial C}{\partial z_j^{l+1}} \frac{\partial z_j^{l+1}}{\partial z_k^l} = \sum_j d_j^{l+1} f^{'} \left( z^l_k\right) w_{kj}^l$

Pre parametre platí:

$\frac{\partial C}{\partial b_j^l} = d_j^l$

$\frac{\partial C}{\partial w_{kj}^l} = d_j^l a_k^{l-1}$

Vektorovo:

$\frac{\partial C}{\partial b^l} = d^l$

$\frac{\partial C}{\partial w_{}^l} = (a^{l-1})^T d^l $

Dôkaz:

$z_j^l = \sum_k a_k^{l-1} w_{kj}^l + b_j^l$

$\frac{\partial C}{\partial b_j^l} = \sum_k \frac{\partial C}{\partial z_k^l}\frac{\partial z_k^l}{\partial b_j^l} = \sum_k \delta_{kj} d_k^l = d_j^l$

$\frac{\partial C}{\partial w_{kj}^l} = \sum_p \frac{\partial C}{\partial z_p^l}\frac{\partial z_p^l}{\partial w_{kj}^l} = \sum_p d_p^l \delta_{pj} a_k^{l-1} = d_j^l a_k^{l-1}$  

kde $\delta_{jk} = 1$ ak $j = k$, inak $ 0$.

### Algoritmus
Jeden krok algoritmu SGD potom vyzerá následovne
1. Realizujeme dopredný výpočet - pamätáme si $z^l$ a $a^l$.
2. Vypočitame $\delta^L$ podľa cenovej funkcie
3. Spätne propagujeme chybu $\delta^l$ pomocou backpropagation pravidla.
4. Vypočítame parciálne derivácie podľa rovníc z predchádzajúceho bloku.
5. Opakujeme pre celý minibatch, derivácie spriemerujeme a updatneme parametre podľa gradientného zostupu.
  
**Pozor:** je nutné ešte nejako nainicializovať váhy. Ak ich nainicializujeme narovnako, tak derivácie pre jednotlivé vrstvy budú vždy rovnaké a tak sa sieť ani nemôže naučit komplikovanejšie reprezentácie. Tj. je to ako sieť kde v každej vrstve je len jeden neurón.

## Implementácia

V nasledujúcom kóde je implementácia základnej štruktúry kódu. Pre jednoduchú plne prepojenú sieť. V kóde budete musieť doplniť veci v jednotlivých úlohách. K úlohám je aj kód, ktorý Vám pomôže overiť či ste správne doimplementovali časť riešenia.

In [0]:
import numpy as np

def sigmoid(z):
    # vráti hodnotu sigmoidu
    ...

def sigmoid_prime(z):
    # vráti gradient sigmoidu
    ...

def get_one_hot(targets, nb_classes):
    # vráti one-hot vektor pre vektor tried v tvare akom ho načítame
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

class Network():
    def __init__(self, arg):
        if isinstance(arg, str):
            # Ak máme na vstup string, tak načítame súbor
            self.load(arg)
        else:
            # Inak čakáme zoznam a podľa neho určíme počty neurónov
            if len(arg) < 2:
                raise ValueError("Sizes must be at least 2!")

            self.w_list = []
            self.b_list = []
            # Je dôležité neuróny nainicializovať
            for i in range(1, len(arg)):
                self.w_list.append(np.random.randn(arg[i - 1], arg[i]))
                self.b_list.append(0.1 * np.ones((arg[i])))

    def save(self, filename):
        # uloženie siete ako numpy súboru
        dict = np.array([self.w_list, self.b_list])
        np.save(filename, dict)

    def load(self, filename):
        # čítanie uložených npy súborov
        d = np.load(filename, allow_pickle=True)
        self.w_list = d[0].tolist()
        self.b_list = d[1].tolist()

    def fwd(self, a):
        # doimplementujte dopredný beh
        ...

    def _step(self, X, y, batch_size, eta):
        # doplne tieto zoznamy podľa SGD kroku na minibatchi
        new_w_list = [np.empty_like(w) for w in self.w_list]
        new_b_list = [np.empty_like(b) for b in self.b_list]

        # najprv bude nutné realizovať dopredný krok a pamätať si pomocné hodnoty
        # je dosť možne, že len(a_list) = len(z_list) + 1, lebo ako prvé a máme
        # vstup treba to potom zohľadniť pri indexovaní môžete to ale urobiť aj
        # inak
        z_list = []
        a_list = [X]
        # a_list = []

        # tu najprv naplníme a_list a z_list

        # vypočítame chybu v danom kroku
        err = 0
        d_list = [[] for _ in z_list]
        # výpočet d^L podľa cenovej funkcie

        # backprop výpočet d

        # zápis do new_w_list a new_b_list podľa SGD

        self.w_list = new_w_list
        self.b_list = new_b_list
        return err

    def eval(self, X, y):
        # táto metóda spustí sieť na dáta X a spočíta presnosť a chybu na týchto
        # dátach
        
        # Pozn: normálne by bolo fajn robiť to po častiach kvôli pamäti. Náš
        # model bude ale malý tak to môžeme spraviť naraz
        out = self.fwd(X)
        err = np.mean((out - y)**2)
        out_best = np.argmax(out, axis=-1)
        y_best = np.argmax(y, axis=-1)
        acc = np.sum(np.where(out_best == y_best, 1, 0))/y.shape[0]
        return acc, err

    def sgd(self, X, y, val_X, val_y, epochs, steps, batch_size, eta):
        for epoch in range(epochs):
            err_tot = 0
            for step in range(steps):
                batch_mask = np.random.choice(X.shape[0], batch_size)
                batch_X = X[batch_mask,:]
                batch_y = y[batch_mask,:]
                err = self._step(batch_X, batch_y, batch_size, eta)
                if step == 0:
                    err_tot = err
                else:
                    err_tot = 0.1 * err + 0.9 * err_tot
                print("At step: {}, running average training error: {}".format(step, err_tot))

            acc, err = self.eval(val_X, val_y)
            print("At Epoch {}, validation error: {}, validation accuracy {}".format(epoch, err, acc))

### Úloha 2

Doimplementujte funkciu sigmoid, ktorá vráti výsledok funkcie sigmoid po elementoch pre každý vstupný tenzor.

$$\sigma(z) = \frac{1}{1 + e^{-z}}$$

Takisto doimplementujte aj deriváciu

$$\sigma (z)^{'} = \sigma (z)\left(1 - \sigma (z) \right)$$



In [0]:
assert(sigmoid(0) == 0.5)
assert(sigmoid(50) > 1.0-1e-10)
assert(sigmoid(-50) <= 2e-22)
assert((sigmoid(np.array([10, 20]))==np.array([sigmoid(10), sigmoid(20)])).all())
assert(sigmoid_prime(0) == 0.25)
assert(sigmoid_prime(-50) > 0 )
assert(sigmoid_prime(30) > 0 )

### Úloha 3

Pred treťou úlohou načítame mnist dataset a zobrazíme si jeden obrázok. Je dosť možné, že to chvílu potrvá.

In [0]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

X = mnist.data.astype('float32')/255
y = mnist.target.astype('int64')

In [0]:
import matplotlib.pyplot as plt

plt.imshow(np.reshape(X[0,],(28,28)), cmap='gray')
print(y[0])

Doimplementujte metódu fwd. V prvom teste sa testuje dimenzia výstupu a v druhuom sa stihane predtrénovaná sieť a spustí sa na obrázkoch z datasetu MNIST.

In [0]:
net = Network([28*28,30,20,10])
out = net.fwd(np.random.randn(32,784))
assert(out.shape == (32,10))
print("Prvý test prešiel")

In [0]:
!wget https://github.com/kocurvik/edu/raw/master/PNNPPV/supplementary/test_net.npy

In [0]:
net = Network("test_net.npy")

R = net.fwd(X[0:3,:])

correct_list = []

for i in range(3):
  plt.imshow(np.reshape(X[i,:],(28,28)), cmap = 'gray')
  plt.show()
  print(R[i])
  print(np.argmax(R[i,:]))
  
print("Druhý test prešiel!")

### Úloha 4 
Doimplementujte metódu _step

Ktorá výkoná krok SGD pre MSE, alebo CE cenovú funkciu. Túto metódu nebude testovať priamo, ale skúsime je apllikovať na trénovanie. Tieto parametre by mali fungovať aspoň tak, že model sa bude zlepšovať na validačnej množine.

In [0]:
train_X = X[:50000, :]
train_y = get_one_hot(y[:50000], 10)


val_X = X[50000:60000,:]
val_y = get_one_hot(y[50000:60000],10)
#
net = Network([28*28,30,20,10])
net.sgd(train_X, train_y, val_X, val_y, 10, 10000, 32, 0.01)
net.sgd(train_X, train_y, val_X, val_y, 10, 10000, 32, 0.03)
net.sgd(train_X, train_y, val_X, val_y, 10, 10000, 32, 0.001)
net.save("net.npy")

Môžeme si sieť aj otestovať!

In [0]:
R = net.fwd(X[60000:60010,:])

for i in range(10):
  plt.imshow(np.reshape(X[60000 + i,:],(28,28)), cmap = 'gray')
  plt.show()
  print(R[i])
  print(np.argmax(R[i,:]))